# Compiling Python to C with Cython
Brian Svoboda --- Jansky Fellow, National Radio Astronomy Observatory

Cython is a compiler that *transforms Python into C*. The C code is then compiled into a library that one may import as a Python module: ``import mylib; mylib.my_c_func(5)``. It is useful for an astronomer who knows Python but who needs to write a new high-performance algorithm or create an interface to legacy C/C++ code.


Cython is well-suited for scientists:
* Write new efficient numerical code that execute at C speed
* Wrap existing C/C++ libraries directly and call them from Python
* Easy to learn: it's mostly Python!
* Mature, well-tested software distribution with detailed documentation. Ready to use now!


However, drawbacks include:
* To wrap C and write fast algorithms, you must partially understand C.
* For non-trivial programs you must use a build-system (``setup.py``).


**Resources:**
* Home: http://cython.org
* Documentation: http://docs.cython.org
* "Cython: A guide for Python programmers" by Kurth Smith

## What is Python?
Python (the language) is implemented as a C program, CPython. CPython transforms a source code file ".py" into abstract instructions for a virtual machine (VM) ".pyc". These "bytecode" instructions are interpreted by the VM and dispatched to concrete implementations written in C so they can run on a real machine, your computer.

The example below is adapted from Brandon Rhodes Pycon 2014 talk "The day of the EXE is upon us".

In [7]:
import dis

def mult_two(x, y):
    return x * y

dis.dis(mult_two)

  3           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_MULTIPLY
              6 RETURN_VALUE


The CPython VM is a stack machine. This is common and used by Java and PostScript. Stack machines were brought to prominence by then NRAO software developer Chuck Moore when he developed the programming language ``FORTH`` for the NRAO 36 ft telescope control software (Moore 1974b "The FORTH Program for Spectral Line Observing on NRAO's 36 ft Telescope", ApJS, 15, 3).


```
stack    operation            comment
-----    ---------            -------
[]       //                   // stack is empty
[2]      // LOAD_FAST 0 (x)   // push `x` (2) onto top of stack
[2, 3]   // LOAD_FAST 1 (y)   // push `y` (3) onto top of stack
[6]      // BINARY_MULTIPLY   // pop two, multiply, and push result
[]       // RETURN_VALUE      // pop top of stack and return
```

The CPython interpreter is simply a big switch/case statement on these instructions.

```c
// ceval.c
for (;;) {
    ...
    opcode = NEXTOP();
    ...
    switch(opcode) {
    TARGET(NOP) ...
    TARGET(LOAD_FAST) ...
    TARGET(BINARY_MULTIPLY) ...
        PyNumber_Multiply(left, right); ...
    TARGET(RETURN_VALUE)
    }
    ...
}
```

CPython is dynamic: the VM must inspect objects at run-time to avoid unsafe execution. **CPython is slow because it is dynamic**: ``PyNumber_Multiply`` is complex and expensive even though it is written in C. ``PyNumber_Multiply`` must check if both arguments have a multiply method associated with their types and then call the appropriate function.

CPython is implemented with only one type, a pointer (memory address) to a PyObject: ``PyObject *``. The PyObject struct contains a pointer to the type specification, which defines what operations can be done on that type (i.e., multiplication for floating point numbers).

```c
// abstract.c
PyObject *
PyNumber_Multiply(PyObject *v, PyObject *w) {
    PyObject *result = binary_op1(
        v, w, NB_SLOT(nb_multiply));
    ...
    return result;
}
```

```c
// floatobject.c
static PyNumberMethods float_as_number = {
    float_add,      /*nb_add*/
    float_sub,      /*nb_subtract*/
    float_mul,      /*nb_multiply*/
    ...
}
```

```c
// floatobject.c
static PyObject *
float_mul(PyObject *v, PyObject *w) {
    double a, b;
    CONVERT_TO_DOUBLE(v, a);
    CONVERT_TO_DOUBLE(w, b);
    PyFPE_START_PROTECT("multiply", return 0)
    a = a * b;  // we did it!!
    PyFPE_END_PROTECT(a)
    return PyFloat_FromDouble(a);
}
```

The C syntax `a = a * b;` can then be compiled by GCC into instructions that finally do something:

```
        movsd   xmm0, QWORD PTR [rbp-8]
        mulsd   xmm0, QWORD PTR [rbp-16]
        movsd   QWORD PTR [rbp-8], xmm0
```

These assembly instructions are converted to numbers (machine code) that the transistors composing the CPU will receive through electrical signals to perform actual computation:
```
        f2      0f  10  45  f8
        f2      0f  59  45  f0
        f2      0f  11  45  f8
```

Through all these levels of indirection, we have finally arrived at the answer `2 * 3 = 6`. This bare number is wrapped into a Python object and bubbled back up through CPython to be used further.

## Why is optimizing Python hard?
Python's dynamicism makes it simple, fast to develop, and **extremely flexible**, but obfuscates optimization. We must guarentee that variable types are stable to create efficient execution (eg. always ``int`` or ``double``).

## Optimizing Python code

Projects relevant for science optimize code by adding guarentees that types are stable:
* ``numpy``: call C functions that operate on dense arrays of static types
* ``Cython``: compile a dialect of Python to C
* ``numba``: incrementally compile a numeric Python function at run-time

## Cython: compile Python to C
* Adds new syntax to Python to reflect C semantics: types, pointers, structs, etc.
* Compile code to straight C, otherwise call Python's C API (``PyNumber_Multiply``).
* Compile the C code with an optimizing compiler (eg. gcc) to generate fast machine code.

In [1]:
%load_ext Cython

## Accelerating numerical code
We avoid dynamic code execution by explicitly declaring types such as ``int``, ``long``, and ``double``.

In [26]:
def py_sum(n=10000):
    v = 0
    for i in range(n):
        v += i * i + i
    return v

In [27]:
%%cython -a --compile-args=-march=native

def cy_sum(long n=10000):
    cdef long i, v
    v = 0
    for i in range(n):
        v += i * i + i
    return v

In [31]:
print(py_sum(1000))
print(cy_sum(1000))

333333000
333333000


In [61]:
%timeit py_sum()

811 µs ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [79]:
%timeit cy_sum()

4.22 µs ± 34.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [34]:
# Always think about your algorithms!
def fastest_sum(n=10000):
    n -= 1
    return n * (n + 1) * (n + 2) // 3
fastest_sum(1000)

333333000

In [36]:
%timeit fastest_sum()

227 ns ± 0.697 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Extension types
Cython allows you to define your own types through ``cdef class``'s. These declare the types of their attributes and then allocate their memory in an internal struct. These are very useful abstractions for writing readable and maintable code.

In [22]:
%%cython

cdef class Shrubbery:
    cdef int width, height
    
    def __init__(self, w, h):
        self.width = w
        self.height = h
        
    def describe(self):
        print(f'This shrubbery is {self.width} by {self.height}')
    
    
def widen_shrubbery(Shrubbery sh, extra_width):
    sh.width = sh.width + extra_width   

In [20]:
quest = Shrubbery(2, 3)
widen_shrubbery(quest, 3)
quest.describe()

This shrubbery is 5 by 3


## Calling external C code
Calling external C code is straight forward, essentially take the line from the C header file to create a function declaration.

In [53]:
%%cython

cdef extern from 'math.h':
    double exp(double)
    
def call_exp(x):
    return exp(x)

In [54]:
import math

print(math.exp(5.0))
print(call_exp(5.0))

148.4131591025766
148.4131591025766


## Is this better?
* ``numpy`` algorithms must be expressed as multi-dimensional vectors to avoid for-loops through "broadcasting" (vector outer-products) and then reduction along an axis. It can be difficult to express algorithms this way.


* ``numba`` is limited to simple algorithms and a sub-set of mathematical functions (``sin``, ``exp``). It is simple to use, but it does not scale well to complex programs.


* Wrapping C directly with ``ctypes`` or ``SWIG`` can be onerous. This is especially true if the C functions must interact with Python objects (ie, a ``numpy`` array).


**Cython allows seamless scaling between the highest Python to the lowest C and everywhere in between --- and all within the same file or code base!**

## An example in the real world
The nested sampling Monte Carlo code MultiNest is written in Fortran and exposes a C interface. The exposed ``run`` function takes a set of parameters and two functions: one two calculate the likelihood at each iteration and another to dump the calculated values. These functions will be given pointers, not values. We want to efficiently store the values into an HDF5 data file.

We use Cython to create a C function ``mn_dump`` to pass to MultiNest but use Python's ``h5py`` module to handle the HDF5 file. We also create a numpy array wrapper around the memory without copying the values.

In [ ]:
%%cython

import h5py

# ...

cdef void mn_dump(int *n_samples, int *n_live, int *n_params,
            double **phys_live, double **posterior, double **param_constr,
            double *max_loglike, double *lnZ, double *ins_lnZ,
            double *lnZ_err, void *context):
    # ...
    with h5py.File(dumper.store_name, 'a') as hdf:
        group = hdf.create_group(dumper.group_name)
        # nested sampling attributes:
        group.attrs['n_samples']      = n_samples[0]
        group.attrs['n_live']         = n_live[0]
        group.attrs['n_params']       = n_params[0]
        # ...
        # posterior samples
        post_shape = (n_samples[0], n_params[0]+2)
        post_arr = mn_dptr_to_ndarray(posterior, post_shape)
        group.create_dataset('posteriors', data=post_arr)
        group.create_dataset('marginals', data=dumper.calc_marginals(post_arr))
        # ...

## Using Cython in libraries
One of Cython's primary difficulties is that it requires a build system (a ``setup.py`` with ``distutils`` commands), and most Python modules are single file scripts. If you have a build system in place, then it is quite easy, but even without one, simple Cython modules can be used with ``pyximport``.

In [ ]:
# file `mylib.pyx` is in the same directory

import pyximport
pyximport.install()
# ^ this cythonizes the `.pyx` into `.c`, compiles the C code,
# and creates a shared object `.so` library in the current directory
# that may be imported by Python.

import mylib
from mylib import my_other_func

print(mylib.my_compiled_func(2))

## Concluding remarks
Cython is a versatile and mature tool for numeric computing in Python. It is straight forward to learn if you know Python and can be used to interface with existing C/C++ as well as write new high-performance code in a simpler to maintain language.
